# 트랜스포머 인코더 시각화: 위치 인코딩

이 노트북은 `manim` 라이브러리를 사용하여 트랜스포머 모델 인코더의 위치 인코딩(Positional Encoding) 단계를 시각화합니다.

In [7]:
# 필요한 라이브러리 임포트
import numpy as np
from manim import *

# manim 환경 설정 (선택 사항, 필요에 따라 조정)
config.background_color = WHITE
config.frame_height = 6
config.frame_width = 10

In [9]:
%%manim -v WARNING -qh PositionalEncodingVisualization

class PositionalEncodingVisualization(Scene):
    def construct(self):
        # 씬 배경색 설정 (노트북 셀 배경과 유사하게)
        self.camera.background_color = WHITE

        # 제목
        title = Text("2. Positional Encoding", color=BLACK).to_edge(UP)
        self.play(Write(title))
        self.wait(0.5)

        # 임베딩 벡터 (이전 단계에서 생성되었다고 가정)
        embedding_dim = 4
        seq_len = 3
        vector_height = 1.0
        vector_width = 0.25

        embeddings = VGroup(*[
            VGroup(*[Rectangle(height=vector_height/embedding_dim, width=vector_width, fill_opacity=0.8, stroke_width=1, color=BLUE)
                     for _ in range(embedding_dim)]).arrange(DOWN, buff=0)
            for _ in range(seq_len)
        ]).arrange(RIGHT, buff=0.8).shift(UP*0.5)
        embedding_label = Text("Input Embeddings", color=BLACK, font_size=24).next_to(embeddings, UP, buff=0.3)

        self.play(FadeIn(embeddings), Write(embedding_label))
        self.wait(0.5)

        # 위치 인코딩 벡터 생성 (단순화된 시각화)
        # 실제로는 sin/cos 함수 사용
        positional_encodings = VGroup(*[
            VGroup(*[Rectangle(height=vector_height/embedding_dim, width=vector_width, fill_opacity=0.8, stroke_width=1, color=interpolate_color(GREEN, RED, i/seq_len))
                     for _ in range(embedding_dim)]).arrange(DOWN, buff=0)
            for i in range(seq_len)
        ]).arrange(RIGHT, buff=0.8).next_to(embeddings, DOWN, buff=1.5)
        pe_label = Text("Positional Encodings", color=BLACK, font_size=24).next_to(positional_encodings, UP, buff=0.3)

        # 위치 인코딩 벡터 등장 애니메이션
        pe_creation_anims = []
        for i, pe_vec in enumerate(positional_encodings):
            pos_text = Text(f"pos={i}", color=BLACK, font_size=18).next_to(pe_vec, DOWN, buff=0.2)
            pe_creation_anims.extend([FadeIn(pe_vec, shift=DOWN*0.2), Write(pos_text)])

        self.play(Write(pe_label))
        self.play(LaggedStart(*pe_creation_anims, lag_ratio=0.7))
        self.wait(0.5)

        # 덧셈 기호
        plus_signs = VGroup(*[MathTex("+", color=BLACK).scale(1.5).move_to(
            (embeddings[i].get_center() + positional_encodings[i].get_center()) / 2
        ) for i in range(seq_len)])

        self.play(LaggedStart(*[Write(p) for p in plus_signs], lag_ratio=0.5))
        self.wait(0.5)

        # 결과 벡터 (임베딩 + 위치 인코딩)
        result_embeddings = VGroup(*[
             VGroup(*[Rectangle(height=vector_height/embedding_dim, width=vector_width, fill_opacity=0.8, stroke_width=1, color=interpolate_color(BLUE, positional_encodings[i][0].get_color(), 0.5))
                     for _ in range(embedding_dim)]).arrange(DOWN, buff=0)
            for i in range(seq_len)
        ]).arrange(RIGHT, buff=0.8).next_to(positional_encodings, DOWN, buff=1.5)
        result_label = Text("Embeddings + Positional Encodings", color=BLACK, font_size=24).next_to(result_embeddings, UP, buff=0.3)

        # 결과 벡터 생성 애니메이션 (이동 및 결합)
        transform_anims = []
        for i in range(seq_len):
            # 각 벡터 그룹 복사
            emb_copy = embeddings[i].copy()
            pe_copy = positional_encodings[i].copy()
            # 결과 위치로 이동 및 결과 벡터로 변환
            transform_anims.append(Transform(emb_copy, result_embeddings[i]))
            transform_anims.append(Transform(pe_copy, result_embeddings[i])) # PE도 결과로 변환

        self.play(Write(result_label))
        # 원본 임베딩과 PE는 FadeOut, 복사본은 Transform
        self.play(
            LaggedStart(*transform_anims, lag_ratio=0.1),
            FadeOut(embeddings),
            FadeOut(positional_encodings),
            FadeOut(embedding_label),
            FadeOut(pe_label),
            FadeOut(VGroup(*[p.mobject for p in pe_creation_anims[1::2]])), # pos=i 텍스트 FadeOut (오류 수정)
            FadeOut(plus_signs)
        )
        # 결과 벡터 그룹을 중앙으로 이동
        self.play(result_embeddings.animate.move_to(ORIGIN), result_label.animate.next_to(result_embeddings, UP, buff=0.3))

        self.wait(2)

Manim Community v0.19.0

In [10]:
%%manim -v WARNING -qh CircleAreaCalculation

class CircleAreaCalculation(Scene):
    def construct(self):
        # 원 생성
        circle = Circle(radius=2)
        circle.set_fill(BLUE, opacity=0.5)

        # 텍스트 생성
        radius_text = MathTex(r"r = 2")
        area_formula = MathTex(r"A = \pi r^2")
        area_result = MathTex(r"A = \pi \cdot 2^2 = 4\pi \approx 12.57")

        # 텍스트 배치
        radius_text.next_to(circle, DOWN)
        VGroup(area_formula, area_result).arrange(DOWN).next_to(circle, RIGHT)

        # 애니메이션 실행
        self.play(Create(circle))
        self.play(Write(radius_text))
        self.wait(1)
        self.play(Write(area_formula))
        self.wait(1)
        self.play(Write(area_result))
        self.wait(2)


Manim Community v0.19.0

In [14]:
%%manim -v WARNING -qh ExpWithBars

class ExpWithBars(Scene):
    def construct(self):
        # Title
        title = Text("Visualizing exp(score) in Attention", font_size=36, color=YELLOW)
        title.to_edge(UP, buff=0.5)
        self.play(Write(title))
        self.wait(0.5)

        # Narration 1
        narration1 = Paragraph(
            "We start with raw attention scores from the dot product of Query and Key.",
            font_size=24, width=10
        ).to_edge(DOWN, buff=0.5)
        self.play(Write(narration1))
        self.wait(1)

        # Raw scores
        raw_scores = [-1, 0, 1]
        bar_colors = [BLUE, ORANGE, GREEN]

        raw_bars = VGroup()
        raw_labels = VGroup()
        for i, score in enumerate(raw_scores):
            height = score + 2  # shift to make -1 visible (base = 1)
            rect = Rectangle(width=0.5, height=height, color=bar_colors[i], fill_opacity=0.7)
            rect.shift(DOWN * (1 - height / 2))  # adjust so all bars start at y= -1
            label = MathTex(str(score)).scale(0.8).next_to(rect, DOWN, buff=0.2)
            group = VGroup(rect, label)
            raw_bars.add(group)

        raw_bars.arrange(RIGHT, buff=1)
        raw_bars.shift(UP * 0.5)

        raw_label = Text("Raw Scores", font_size=28).next_to(raw_bars, UP, buff=0.3)
        self.play(Write(raw_label), Create(raw_bars))
        self.wait(1.5)

        # Narration 2
        self.play(FadeOut(narration1))
        narration2 = Paragraph(
            "Now we apply the exponential function to each score.",
            font_size=24, width=10
        ).to_edge(DOWN, buff=0.5)
        self.play(Write(narration2))
        self.wait(1.5)

        # exp(score) -> new heights
        exp_scores = [np.exp(s) for s in raw_scores]
        max_height = max(exp_scores)

        exp_bars = VGroup()
        exp_labels = VGroup()
        for i, val in enumerate(exp_scores):
            height = (val / max_height) * 3  # Normalize to max height = 3 for viewport
            rect = Rectangle(width=0.5, height=height, color=bar_colors[i], fill_opacity=0.7)
            rect.shift(DOWN * (1 - height / 2))  # anchor bottom at y = -1
            label = MathTex(r"\exp(" + str(raw_scores[i]) + r") = " + str(round(val, 2))).scale(0.8)
            label.next_to(rect, DOWN, buff=0.2)
            group = VGroup(rect, label)
            exp_bars.add(group)

        exp_bars.arrange(RIGHT, buff=1)
        exp_bars.shift(DOWN * 1.2)

        exp_label = Text("After exp(score)", font_size=28).next_to(exp_bars, UP, buff=0.3)

        # Animate transform from raw to exp
        self.play(Transform(raw_label, exp_label), Transform(raw_bars, exp_bars))
        self.wait(2)

        # Narration 3
        self.play(FadeOut(narration2))
        narration3 = Paragraph(
            "The exponential highlights large scores while shrinking smaller ones.",
            font_size=24, width=10
        ).to_edge(DOWN, buff=0.5)
        self.play(Write(narration3))
        self.wait(2)

        # Fade everything out
        self.play(
            FadeOut(title),
            FadeOut(exp_label),
            FadeOut(raw_bars),
            FadeOut(narration3)
        )
        self.wait()




Manim Community v0.19.0

In [20]:
%%manim -v WARNING -qh RectangleColumns

class RectangleColumns(Scene):
    def construct(self):
        self.camera.background_color = WHITE # 배경 흰색으로 설정

        square_side = 0.8 # 정사각형 변 길이
        num_squares = 4
        column_buff = 0.2 # 열 내 사각형 간 간격
        inter_column_buff = 3 # 열 간 간격

        # 왼쪽 열 색상 (파란색 계열)
        left_colors = [BLUE_E, BLUE_D, BLUE_C, BLUE_B]
        # 오른쪽 열 색상 (초록색 계열)
        right_colors = [GREEN_E, GREEN_D, GREEN_C, GREEN_B]

        # 왼쪽 열 생성 (정사각형으로 변경)
        left_column = VGroup(*[
            Square(side_length=square_side, color=left_colors[i], fill_opacity=0.8, stroke_color=BLACK, stroke_width=2) # 테두리 추가
            for i in range(num_squares)
        ]).arrange(DOWN, buff=column_buff)

        # 오른쪽 열 생성 (정사각형으로 변경)
        right_column = VGroup(*[
            Square(side_length=square_side, color=right_colors[i], fill_opacity=0.8, stroke_color=BLACK, stroke_width=2) # 테두리 추가
            for i in range(num_squares)
        ]).arrange(DOWN, buff=column_buff)

        # 전체 그룹 생성 및 배치
        columns = VGroup(left_column, right_column).arrange(RIGHT, buff=inter_column_buff)

        # 애니메이션
        self.play(Create(columns))
        self.wait(1) # 초기 대기 시간

        # --- 첫 번째 정사각형 분할 및 이동 ---
        source_square = left_column[0]
        source_color = source_square.get_color()
        small_square_side = square_side / 2

        # 작은 정사각형 4개 생성 (초기 위치는 원본 정사각형 내부)
        small_squares = VGroup()
        positions = [
            source_square.get_center() + UL * small_square_side / 2,
            source_square.get_center() + UR * small_square_side / 2,
            source_square.get_center() + DL * small_square_side / 2,
            source_square.get_center() + DR * small_square_side / 2,
        ]
        for pos in positions:
            small_sq = Square(side_length=small_square_side, color=source_color, fill_opacity=0.8, stroke_color=BLACK, stroke_width=1)
            small_sq.move_to(pos)
            small_squares.add(small_sq)

        # 원본 정사각형을 작은 정사각형 4개로 변환 (분할 효과)
        self.play(ReplacementTransform(source_square, small_squares), run_time=1)
        self.wait(0.5)

        # 작은 정사각형들을 오른쪽 열의 각 정사각형으로 이동
        move_anims = []
        for i in range(num_squares):
            move_anims.append(small_squares[i].animate.move_to(right_column[i].get_center()))

        self.play(LaggedStart(*move_anims, lag_ratio=0.2), run_time=1.5)
        self.wait(0.5)

        # 작은 정사각형들 사라짐
        self.play(FadeOut(small_squares))

        self.wait(2) # 최종 대기 시간

Manim Community v0.19.0

In [24]:
%%manim -v WARNING -qh QuickSortVisualization

class QuickSortVisualization(Scene):
    def construct(self):
        # 배경색을 흰색으로 설정
        self.camera.background_color = WHITE
        
        # 제목 생성
        title = Text("Quick Sort", font_size=36, color=BLACK).to_edge(UP)
        self.play(Write(title))
        
        # 배열 크기와 막대 설정
        n = 20  # 더 많은 막대를 원할 경우 조정 (100개는 너무 많아 보기 어려울 수 있음)
        bar_width = 0.25
        max_height = 4
        spacing = bar_width * 1.2
        
        # 정렬되지 않은 랜덤 데이터 생성
        np.random.seed(42)  # 일관된 결과를 위해 시드 설정
        data = np.random.randint(10, 100, n)
        
        # 기준선 생성 (모든 막대가 위치할 밑변 라인)
        base_line = Line(
            start=LEFT * (n * spacing / 2),
            end=RIGHT * (n * spacing / 2),
            color=GREY
        ).shift(DOWN * 1.5)
        
        # 막대 그래프 생성 - 밑변을 맞추어 생성
        bars = VGroup()
        for i, val in enumerate(data):
            height = val / 100 * max_height  # 높이 정규화
            bar = Rectangle(
                height=height, 
                width=bar_width, 
                fill_opacity=0.8, 
                stroke_width=1,
                stroke_color=BLACK,
                fill_color=BLUE
            )
            # 막대의 밑면을 기준선에 맞추어 정렬
            bar.align_to(base_line, DOWN)
            bar.shift(RIGHT * (i * spacing - n * spacing / 2 + bar_width / 2))
            bars.add(bar)
        
        # 초기 배열 그리기
        self.play(Create(base_line, run_time=0.5), FadeIn(bars))
        
        # 초기 상태 설명 (간결하게)
        step_text = Text("Unsorted Array", font_size=28, color=BLACK)
        step_text.next_to(title, DOWN)
        self.play(Write(step_text))
        self.wait(1)
        
        # 퀵소트 함수 구현
        def quick_sort(start_idx, end_idx, depth=0):
            if start_idx >= end_idx:
                return

            # 피벗 선택 (마지막 요소)
            pivot_idx = end_idx
            pivot_val = data[pivot_idx]
            
            # 피벗 강조
            pivot_bar = bars[pivot_idx]
            if depth == 0:
                self.play(
                    pivot_bar.animate.set_fill(RED),
                    FadeOut(step_text)
                )
            else:
                self.play(
                    pivot_bar.animate.set_fill(RED)
                )
            
            i = start_idx - 1  # i는 작은 요소들의 경계를 추적
            
            for j in range(start_idx, end_idx):
                # 현재 비교 중인 요소 강조
                compare_bar = bars[j]
                self.play(compare_bar.animate.set_fill(YELLOW), run_time=0.3)
                
                if data[j] <= pivot_val:
                    i += 1
                    # i와 j 위치 교환 (필요한 경우)
                    if i != j:
                        # 데이터 교환
                        data[i], data[j] = data[j], data[i]
                        
                        # 막대 교환 애니메이션
                        bar_i = bars[i]
                        bar_j = bars[j]
                        
                        # 교환 강조
                        self.play(
                            bar_i.animate.set_fill(GREEN),
                            bar_j.animate.set_fill(GREEN),
                            run_time=0.3
                        )
                        
                        # x 위치 계산 - 막대의 높이와 관계없이 x좌표만 교환
                        x_i = bar_i.get_center()[0]
                        x_j = bar_j.get_center()[0]
                        
                        # 애니메이션으로 교환 - 밑변을 유지하며 이동
                        self.play(
                            bar_i.animate.shift(RIGHT * (x_j - x_i)),
                            bar_j.animate.shift(RIGHT * (x_i - x_j)),
                            run_time=0.6
                        )
                        
                        # 참조도 교환
                        bars[i], bars[j] = bars[j], bars[i]
                        
                # 색상 원래대로
                self.play(
                    compare_bar.animate.set_fill(BLUE),
                    run_time=0.2
                )
                if i >= 0 and i != j:  # 교환이 발생했을 때만 색상 복원
                    self.play(
                        bars[i].animate.set_fill(BLUE),
                        run_time=0.2
                    )
            
            # 피벗을 올바른 위치에 배치
            i += 1
            if i != pivot_idx:
                # 데이터 교환
                data[i], data[pivot_idx] = data[pivot_idx], data[i]
                
                # 막대 교환 애니메이션
                bar_i = bars[i]
                bar_pivot = bars[pivot_idx]
                
                # x 위치 계산 - 막대의 높이와 관계없이 x좌표만 교환
                x_i = bar_i.get_center()[0]
                x_pivot = bar_pivot.get_center()[0]
                
                # 애니메이션으로 교환 - 밑변을 유지하며 이동
                self.play(
                    bar_i.animate.shift(RIGHT * (x_pivot - x_i)),
                    bar_pivot.animate.shift(RIGHT * (x_i - x_pivot)),
                    run_time=0.6
                )
                
                # 참조도 교환
                bars[i], bars[pivot_idx] = bars[pivot_idx], bars[i]
            
            # 모든 막대 색상 복원
            self.play(*[bar.animate.set_fill(BLUE) for bar in bars], run_time=0.5)
            
            # 재귀 호출
            quick_sort(start_idx, i - 1, depth + 1)  # 왼쪽 부분 배열 정렬
            quick_sort(i + 1, end_idx, depth + 1)  # 오른쪽 부분 배열 정렬
            
            # 가장 상위 호출이 완료된 후 정렬 완료 메시지 표시
            if depth == 0:
                final_text = Text("Sorted!", font_size=32, color=BLACK)
                final_text.next_to(title, DOWN)
                self.play(Write(final_text))
                
                # 모든 막대를 녹색으로 변경하여 완료 표시
                self.play(*[bar.animate.set_fill(GREEN) for bar in bars], run_time=1)
        
        # 퀵소트 호출
        quick_sort(0, n - 1)
        
        self.wait(2)

Manim Community v0.19.0

In [25]:
%%manim -v WARNING -qh BinaryTreeTraversal

class BinaryTreeTraversal(Scene):
    def construct(self):
        # 배경색을 흰색으로 설정
        self.camera.background_color = WHITE
        
        # 이진 트리 생성 및 애니메이션 구현
        self.scene1_intro()
        self.scene2_preorder()
        self.scene3_inorder()
        self.scene4_postorder()
        self.scene5_comparison()
    
    def create_node(self, value, position=ORIGIN):
        """노드 생성 함수"""
        circle = Circle(radius=0.4, color=BLACK, stroke_width=2, fill_opacity=0.3, fill_color=BLUE_C)
        text = Text(str(value), color=BLACK, font_size=30)
        node = VGroup(circle, text).move_to(position)
        return node
    
    def scene1_intro(self):
        # 제목
        title = Text("이진 트리 구조", color=BLACK, font_size=48).to_edge(UP)
        intro_text = Text("이진 트리", color=BLACK, font_size=60)
        
        # 트리 구성 시작
        self.play(Write(intro_text))
        self.wait(1)
        self.play(ReplacementTransform(intro_text, title))
        
        # 트리 구성
        # 레벨 간 간격과 노드 간 간격 설정
        level_spacing = 1.2
        node_spacing = 1.5
        
        # 트리의 위치를 설정 (중심과 간격 등)
        root_pos = DOWN * 0.5
        
        # 각 레벨별 x 오프셋 계산
        level1_xoffset = node_spacing
        level2_xoffset = node_spacing / 2
        
        # 노드 생성
        root = self.create_node(1, root_pos)
        
        level1_left = self.create_node(2, root_pos + DOWN * level_spacing + LEFT * level1_xoffset)
        level1_right = self.create_node(3, root_pos + DOWN * level_spacing + RIGHT * level1_xoffset)
        
        level2_ll = self.create_node(4, level1_left.get_center() + DOWN * level_spacing + LEFT * level2_xoffset)
        level2_lr = self.create_node(5, level1_left.get_center() + DOWN * level_spacing + RIGHT * level2_xoffset)
        level2_rl = self.create_node(6, level1_right.get_center() + DOWN * level_spacing + LEFT * level2_xoffset)
        level2_rr = self.create_node(7, level1_right.get_center() + DOWN * level_spacing + RIGHT * level2_xoffset)
        
        # 에지(간선) 생성
        edges = VGroup()
        edges.add(Line(root.get_center(), level1_left.get_center(), color=BLACK))
        edges.add(Line(root.get_center(), level1_right.get_center(), color=BLACK))
        edges.add(Line(level1_left.get_center(), level2_ll.get_center(), color=BLACK))
        edges.add(Line(level1_left.get_center(), level2_lr.get_center(), color=BLACK))
        edges.add(Line(level1_right.get_center(), level2_rl.get_center(), color=BLACK))
        edges.add(Line(level1_right.get_center(), level2_rr.get_center(), color=BLACK))
        
        # 노드들을 그룹으로 묶어서 전체 트리 구성
        self.nodes = VGroup(root, level1_left, level1_right, level2_ll, level2_lr, level2_rl, level2_rr)
        self.tree = VGroup(edges, self.nodes)
        
        # 루트 노드 등장 애니메이션
        self.play(Create(root))
        self.wait(0.5)
        
        # 1레벨 노드와 간선 등장
        self.play(
            Create(edges[0]), Create(edges[1]),
            AnimationGroup(
                FadeIn(level1_left, shift=UP),
                FadeIn(level1_right, shift=UP),
                lag_ratio=0.2
            )
        )
        self.wait(0.5)
        
        # 2레벨 노드와 간선 등장
        self.play(
            Create(edges[2]), Create(edges[3]), 
            Create(edges[4]), Create(edges[5]),
            AnimationGroup(
                FadeIn(level2_ll, shift=UP), 
                FadeIn(level2_lr, shift=UP),
                FadeIn(level2_rl, shift=UP), 
                FadeIn(level2_rr, shift=UP),
                lag_ratio=0.1
            )
        )
        self.wait(1)
        
        # 트리를 약간 축소하고 중앙에 정렬
        self.play(
            self.tree.animate.scale(0.9).shift(UP * 0.5)
        )
        self.wait(1)
        
        # 객체들을 클래스 변수로 저장하여 다른 장면에서 접근할 수 있게 합니다
        self.title = title
        self.edges = edges
    
    def highlight_node(self, node_idx, color=YELLOW, duration=0.5, restore=True, wait_time=0.5):
        """노드 강조 함수"""
        node = self.nodes[node_idx]
        highlight_anim = node[0].animate.set_fill(color).set_fill_opacity(0.8)
        self.play(highlight_anim, run_time=duration)
        self.wait(wait_time)
        
        if restore:
            restore_anim = node[0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
            self.play(restore_anim, run_time=duration)
    
    def scene2_preorder(self):
        # 제목 변경
        traverse_title = Text("현재 순회 방식: 전위 순회 (Preorder)", color=BLACK, font_size=36).to_edge(DOWN)
        self.play(Write(traverse_title))
        self.wait(1)
        
        # 전위 순회 순서: 1-2-4-5-3-6-7
        preorder_sequence = [0, 1, 3, 4, 2, 5, 6]  # 노드 인덱스 (0부터 시작)
        
        # 방문 순서를 기록할 텍스트
        visit_text = Text("방문 순서: ", color=BLACK, font_size=24).next_to(traverse_title, UP)
        self.play(Write(visit_text))
        
        # 전위 순회 시각화
        visit_order = VGroup()
        visit_order.add(Text("1", color=BLACK, font_size=24))
        visit_order[0].next_to(visit_text, RIGHT)
        
        # 첫 번째 노드 강조와 방문 순서 표시
        self.highlight_node(0, wait_time=0.3, restore=False)
        self.play(Write(visit_order[0]))
        
        # 나머지 노드 순회
        for i, node_idx in enumerate(preorder_sequence[1:]):
            # 이전 노드 색상 복원
            if i == 0:
                restore_anim = self.nodes[preorder_sequence[i]][0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
                self.play(restore_anim, run_time=0.3)
            
            # 새 노드 강조
            self.highlight_node(node_idx, wait_time=0.3, restore=False)
            
            # 방문 순서에 추가
            new_num = Text(str(node_idx + 1), color=BLACK, font_size=24)
            new_num.next_to(visit_order[i], RIGHT, buff=0.2)
            visit_order.add(new_num)
            self.play(Write(new_num))
            
            # 마지막이 아니면 이전 노드 복원
            if i < len(preorder_sequence) - 2:
                restore_anim = self.nodes[node_idx][0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
                self.play(restore_anim, run_time=0.3)
        
        # 모든 노드 색상 복원
        self.play(
            *[node[0].animate.set_fill(BLUE_C).set_fill_opacity(0.3) for node in self.nodes]
        )
        self.wait(1)
        
        # 정보 저장
        self.traverse_title = traverse_title
        self.visit_text = visit_text
        self.preorder_visit = visit_order
    
    def scene3_inorder(self):
        # 제목 변경
        inorder_title = Text("현재 순회 방식: 중위 순회 (Inorder)", color=BLACK, font_size=36).to_edge(DOWN)
        self.play(ReplacementTransform(self.traverse_title, inorder_title))
        
        # 중위 순회 순서: 4-2-5-1-6-3-7
        inorder_sequence = [3, 1, 4, 0, 5, 2, 6]  # 노드 인덱스
        
        # 이전 방문 순서 지우기
        self.play(FadeOut(self.preorder_visit))
        
        # 새로운 방문 순서 표시
        inorder_visit = VGroup()
        inorder_visit.add(Text("4", color=BLACK, font_size=24))
        inorder_visit[0].next_to(self.visit_text, RIGHT)
        
        # 첫 번째 노드 강조와 방문 순서 표시
        self.highlight_node(inorder_sequence[0], wait_time=0.3, restore=False)
        self.play(Write(inorder_visit[0]))
        
        # 나머지 노드 순회
        for i, node_idx in enumerate(inorder_sequence[1:]):
            # 이전 노드 색상 복원
            if i == 0:
                restore_anim = self.nodes[inorder_sequence[i]][0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
                self.play(restore_anim, run_time=0.3)
            
            # 새 노드 강조
            self.highlight_node(node_idx, wait_time=0.3, restore=False)
            
            # 방문 순서에 추가
            new_num = Text(str(node_idx + 1), color=BLACK, font_size=24)
            new_num.next_to(inorder_visit[i], RIGHT, buff=0.2)
            inorder_visit.add(new_num)
            self.play(Write(new_num))
            
            # 마지막이 아니면 이전 노드 복원
            if i < len(inorder_sequence) - 2:
                restore_anim = self.nodes[node_idx][0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
                self.play(restore_anim, run_time=0.3)
        
        # 모든 노드 색상 복원
        self.play(
            *[node[0].animate.set_fill(BLUE_C).set_fill_opacity(0.3) for node in self.nodes]
        )
        self.wait(1)
        
        # 업데이트
        self.traverse_title = inorder_title
        self.inorder_visit = inorder_visit
    
    def scene4_postorder(self):
        # 제목 변경
        postorder_title = Text("현재 순회 방식: 후위 순회 (Postorder)", color=BLACK, font_size=36).to_edge(DOWN)
        self.play(ReplacementTransform(self.traverse_title, postorder_title))
        
        # 후위 순회 순서: 4-5-2-6-7-3-1
        postorder_sequence = [3, 4, 1, 5, 6, 2, 0]  # 노드 인덱스
        
        # 이전 방문 순서 지우기
        self.play(FadeOut(self.inorder_visit))
        
        # 새로운 방문 순서 표시
        postorder_visit = VGroup()
        postorder_visit.add(Text("4", color=BLACK, font_size=24))
        postorder_visit[0].next_to(self.visit_text, RIGHT)
        
        # 첫 번째 노드 강조와 방문 순서 표시
        self.highlight_node(postorder_sequence[0], wait_time=0.3, restore=False)
        self.play(Write(postorder_visit[0]))
        
        # 나머지 노드 순회
        for i, node_idx in enumerate(postorder_sequence[1:]):
            # 이전 노드 색상 복원
            if i == 0:
                restore_anim = self.nodes[postorder_sequence[i]][0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
                self.play(restore_anim, run_time=0.3)
            
            # 새 노드 강조
            self.highlight_node(node_idx, wait_time=0.3, restore=False)
            
            # 방문 순서에 추가
            new_num = Text(str(node_idx + 1), color=BLACK, font_size=24)
            new_num.next_to(postorder_visit[i], RIGHT, buff=0.2)
            postorder_visit.add(new_num)
            self.play(Write(new_num))
            
            # 마지막이 아니면 이전 노드 복원
            if i < len(postorder_sequence) - 2:
                restore_anim = self.nodes[node_idx][0].animate.set_fill(BLUE_C).set_fill_opacity(0.3)
                self.play(restore_anim, run_time=0.3)
        
        # 모든 노드 색상 복원
        self.play(
            *[node[0].animate.set_fill(BLUE_C).set_fill_opacity(0.3) for node in self.nodes]
        )
        self.wait(1)
        
        # 업데이트
        self.traverse_title = postorder_title
        self.postorder_visit = postorder_visit
    
    def scene5_comparison(self):
        # 이전 텍스트 제거
        self.play(
            FadeOut(self.traverse_title),
            FadeOut(self.visit_text),
            FadeOut(self.postorder_visit),
            self.tree.animate.scale(0.8).to_edge(UP, buff=0.5)
        )
        
        # 세 가지 순회 방식 요약
        comparison_title = Text("이진 트리 순회 방식 비교", color=BLACK, font_size=36).to_edge(UP)
        
        # 각 순회 방식별 순서 텍스트
        preorder_text = Text("전위 순회 (Preorder): 루트 → 왼쪽 → 오른쪽", color=BLACK, font_size=24)
        inorder_text = Text("중위 순회 (Inorder): 왼쪽 → 루트 → 오른쪽", color=BLACK, font_size=24)
        postorder_text = Text("후위 순회 (Postorder): 왼쪽 → 오른쪽 → 루트", color=BLACK, font_size=24)
        
        preorder_result = Text("1→2→4→5→3→6→7", color=BLUE, font_size=24)
        inorder_result = Text("4→2→5→1→6→3→7", color=GREEN, font_size=24)
        postorder_result = Text("4→5→2→6→7→3→1", color=RED, font_size=24)
        
        # 텍스트 배치
        preorder_group = VGroup(preorder_text, preorder_result).arrange(DOWN, aligned_edge=LEFT)
        inorder_group = VGroup(inorder_text, inorder_result).arrange(DOWN, aligned_edge=LEFT)
        postorder_group = VGroup(postorder_text, postorder_result).arrange(DOWN, aligned_edge=LEFT)
        
        # 전체 정렬
        comparison_group = VGroup(preorder_group, inorder_group, postorder_group).arrange(DOWN, buff=0.5, aligned_edge=LEFT).center()
        
        # 애니메이션으로 보여주기
        self.play(Write(comparison_title))
        self.play(Write(preorder_group))
        self.wait(0.5)
        self.play(Write(inorder_group))
        self.wait(0.5)
        self.play(Write(postorder_group))
        
        # 특징 설명
        feature_text = Text("활용: 전위(복사), 중위(정렬), 후위(계산식)", color=BLACK, font_size=22)
        feature_text.to_edge(DOWN)
        
        self.play(Write(feature_text))
        self.wait(2)

Manim Community v0.19.0

In [ ]:
%%manim -v WARNING -qh FederatedLearningVisualization

class FederatedLearningVisualization(Scene):
    def construct(self):
        # 배경색을 흰색으로 설정
        self.camera.background_color = WHITE
        
        # 비주얼 스타일 정의
        self.define_visual_styles()
        
        # Scene 1: 개념 소개 - 분산된 데이터
        self.scene1_intro()
        
        # Scene 2: 중앙 서버의 모델 배포
        self.scene2_model_distribution()
        
        # Scene 3: 로컬 학습 - 데이터로 모델 업데이트
        self.scene3_local_training()
        
        # Scene 4: 모델 업데이트만 서버로 전송
        self.scene4_model_updates()
        
        # Scene 5: 새로운 모델 다시 배포
        self.scene5_redistribution()
        
        # Scene 6: 요약 애니메이션
        self.scene6_summary()
    
    def define_visual_styles(self):
        # 색상 정의
        self.server_color = BLUE_E
        self.client_colors = [GREEN_D, TEAL_D, GOLD_D, MAROON_D]
        self.model_color = BLUE_C
        self.updated_model_color = PURPLE_C
        self.data_color = GOLD_C
        self.arrow_color = GRAY
        self.text_color = BLACK
        
        # 크기 정의
        self.server_radius = 0.8
        self.client_width = 1.2
        self.client_height = 0.8
        self.model_width = 0.8
        self.model_height = 0.4
        self.data_size = 0.3
    
    def create_client(self, position, index):
        """클라이언트 장치 생성 함수"""
        # 클라이언트 아이콘 (노트북 형태)
        laptop_body = RoundedRectangle(
            width=self.client_width, height=self.client_height, 
            corner_radius=0.1, fill_color=self.client_colors[index], 
            fill_opacity=0.8, stroke_color=self.text_color
        )
        
        # 노트북 화면 부분
        screen = RoundedRectangle(
            width=self.client_width * 0.85, 
            height=self.client_height * 0.7,
            corner_radius=0.05,
            fill_color=WHITE,
            fill_opacity=1,
            stroke_color=self.text_color,
            stroke_width=1
        ).move_to(laptop_body.get_center() + UP * 0.05)
        
        # 클라이언트 라벨
        label = Text(f"Client {chr(65+index)}", font_size=20, color=self.text_color)
        label.next_to(laptop_body, DOWN, buff=0.2)
        
        # 데이터 아이콘 (파일 형태)
        data = self.create_data_icon().scale(0.7)
        data.move_to(screen.get_center())
        
        # 그룹화 및 위치 지정
        client = VGroup(laptop_body, screen, data)
        client.move_to(position)
        
        return VGroup(client, label)
    
    def create_server(self):
        """중앙 서버 생성 함수"""
        server_body = Circle(
            radius=self.server_radius,
            fill_color=self.server_color,
            fill_opacity=0.8,
            stroke_color=self.text_color
        )
        
        # 서버 내부 디테일 추가
        inner_detail = VGroup()
        for i in range(3):
            line = Line(
                start=UP * 0.4, 
                end=DOWN * 0.4,
                stroke_width=4,
                color=WHITE
            ).shift(RIGHT * (i * 0.25 - 0.25))
            inner_detail.add(line)
        inner_detail.move_to(server_body.get_center())
        
        server = VGroup(server_body, inner_detail)
        
        # 서버 라벨
        label = Text("중앙 서버", font_size=24, color=self.text_color)
        label.next_to(server, DOWN, buff=0.2)
        
        return VGroup(server, label)
    
    def create_model_icon(self, color=None):
        """모델 아이콘 생성 함수"""
        if color is None:
            color = self.model_color
            
        model_box = RoundedRectangle(
            width=self.model_width,
            height=self.model_height,
            corner_radius=0.05,
            fill_color=color,
            fill_opacity=0.8,
            stroke_color=self.text_color
        )
        
        # 모델 내부의 신경망 라인 표현
        lines = VGroup()
        num_lines = 3
        for i in range(num_lines):
            line_y = (i - (num_lines-1)/2) * 0.1
            line = Line(
                start=LEFT * 0.3 + UP * line_y,
                end=RIGHT * 0.3 + UP * line_y,
                stroke_width=2,
                color=WHITE
            )
            lines.add(line)
            
        nodes = VGroup()
        for x in [-0.25, 0, 0.25]:
            for y in [-0.1, 0, 0.1]:
                node = Dot(point=[x, y, 0], radius=0.02, color=WHITE)
                nodes.add(node)
        
        model = VGroup(model_box, lines, nodes)
        
        # 모델 라벨
        label = Text("모델", font_size=16, color=WHITE)
        label.move_to(model_box.get_center())
        
        return VGroup(model, label)
    
    def create_data_icon(self):
        """데이터 아이콘 생성 함수"""
        data_box = Rectangle(
            width=self.data_size,
            height=self.data_size * 1.2,
            fill_color=self.data_color,
            fill_opacity=0.8,
            stroke_color=self.text_color
        )
        
        # 파일 모서리 접힌 효과
        fold = Polygon(
            data_box.get_corner(UR),
            data_box.get_corner(UR) + LEFT * 0.15,
            data_box.get_corner(UR) + DOWN * 0.15,
            fill_color=WHITE,
            fill_opacity=1,
            stroke_color=self.text_color
        )
        
        # 파일 내부 선 (텍스트 라인 표현)
        lines = VGroup()
        for i in range(3):
            line_y = -0.1 * i
            line = Line(
                start=data_box.get_left() + RIGHT * 0.05 + UP * 0.3 + UP * line_y,
                end=data_box.get_right() + LEFT * 0.05 + UP * 0.3 + UP * line_y,
                stroke_width=1,
                color=self.text_color
            )
            lines.add(line)
        
        return VGroup(data_box, fold, lines)
    
    def scene1_intro(self):
        # 타이틀 생성 및 표시
        title = Text("연합학습(Federated Learning)", font_size=48, color=self.text_color)
        title.to_edge(UP, buff=0.5)
        
        self.play(Write(title))
        self.wait(1)
        
        # 클라이언트 장치 위치 설정 (원형으로 배치)
        num_clients = 4
        client_positions = []
        radius = 3.0
        for i in range(num_clients):
            angle = i * TAU / num_clients - PI / 4  # 시작 각도 조정
            x = radius * np.cos(angle)
            y = radius * np.sin(angle)
            client_positions.append([x, y, 0])
        
        # 클라이언트 장치 생성
        clients = VGroup()
        for i, pos in enumerate(client_positions):
            client = self.create_client(pos, i)
            clients.add(client)
            
        # 서버 생성
        server = self.create_server()
        server.move_to(ORIGIN)
        
        # 클라이언트와 서버 등장
        self.play(
            LaggedStart(*[FadeIn(client) for client in clients], lag_ratio=0.2),
            run_time=2
        )
        self.wait(0.5)
        
        self.play(FadeIn(server))
        self.wait(1)
        
        # "로컬 데이터 보유" 텍스트 추가
        local_data_texts = VGroup()
        for i, client in enumerate(clients):
            text = Text("로컬 데이터 보유", font_size=16, color=self.text_color)
            client_pos = client[0].get_center()  # 클라이언트 장치의 중심
            
            # 텍스트 위치 조정 (중앙 서버와 반대 방향으로)
            direction = normalize(client_pos - ORIGIN)
            text.next_to(client, direction=direction, buff=0.2)
            local_data_texts.add(text)
        
        self.play(
            LaggedStart(*[FadeIn(text) for text in local_data_texts], lag_ratio=0.2)
        )
        self.wait(1)
        
        # "데이터는 절대 서버로 가지 않음" 시각화
        server_center = self.server[0].get_center()
        lines_and_crosses = VGroup() # 생성된 선과 X 표시를 추적하기 위한 그룹

        for i, client in enumerate(self.clients):
            client_pos = client[0].get_center()

            # 선 그리기
            line = DashedLine(
                start=server_center,
                end=client_pos,
                stroke_width=2,
                color=self.arrow_color
            )
            lines_and_crosses.add(line) # 그룹에 추가

            # 선 생성 애니메이션 실행
            self.play(Create(line), run_time=0.3)

            # 이제 선이 생성된 후에 중간 지점 계산
            midpoint = line.point_from_proportion(0.6)

            # X자로 가로지르기
            cross1 = Line(
                midpoint + UP * 0.3 + LEFT * 0.3,
                midpoint + DOWN * 0.3 + RIGHT * 0.3,
                stroke_width=4,
                color=RED
            )
            cross2 = Line(
                midpoint + UP * 0.3 + RIGHT * 0.3,
                midpoint + DOWN * 0.3 + LEFT * 0.3,
                stroke_width=4,
                color=RED
            )
            cross_group = VGroup(cross1, cross2)
            lines_and_crosses.add(cross_group) # 그룹에 추가

            no_data_text = Text("데이터 이동 없음", font_size=14, color=RED)
            no_data_text.next_to(cross_group, DOWN, buff=0.1)
            lines_and_crosses.add(no_data_text) # 그룹에 추가

            self.play(
                Create(cross1),
                Create(cross2),
                Write(no_data_text),
                run_time=0.5
            )

        self.wait(1)

        # Scene 1의 객체들을 클래스 멤버로 저장
        self.title = title
        self.clients = clients
        self.server = server
        self.local_data_texts = local_data_texts
        self.lines_and_crosses = lines_and_crosses # 생성된 객체 그룹 저장

        # 초기 화면 정리 (다음 씬을 위한 준비)
        # FadeOut할 객체 목록을 명시적으로 지정 (더 안전한 방식)
        objects_to_fadeout = VGroup(self.local_data_texts, self.lines_and_crosses)
        self.play(FadeOut(objects_to_fadeout))
        # self.play(
        #     *[FadeOut(obj) for obj in self.mobjects if obj not in [title, clients, server]]
        # ) # 이전 방식 대신 위 방식으로 변경
        self.wait(0.5)
    
    def scene2_model_distribution(self):
        # 제목 업데이트
        new_title = Text("1. 초기 모델 배포", font_size=40, color=self.text_color)
        new_title.to_edge(UP, buff=0.5)
        
        self.play(
            Transform(self.title, new_title)
        )
        self.wait(0.5)
        
        # 서버에 모델 아이콘 생성
        server_model = self.create_model_icon()
        server_model.move_to(self.server[0].get_center())
        server_model.scale(0.8)  # 서버 안에 맞게 크기 조정
        
        self.play(FadeIn(server_model))
        self.wait(0.5)
        
        # 모델을 각 클라이언트로 배포하는 애니메이션
        arrows = VGroup()
        model_copies = VGroup()
        
        server_center = self.server[0].get_center()
        for i, client in enumerate(self.clients):
            client_center = client[0].get_center()
            
            # 화살표 생성
            arrow = Arrow(
                start=server_center,
                end=client_center,
                buff=self.server_radius,  # 서버 크기만큼 시작점 조정
                stroke_width=2,
                color=self.arrow_color
            )
            arrows.add(arrow)
            
            # 이동할 모델 복사본 생성
            model_copy = self.create_model_icon().scale(0.7)
            model_copy.move_to(server_center)
            model_copies.add(model_copy)
        
        # 화살표 등장
        self.play(LaggedStart(*[GrowArrow(arrow) for arrow in arrows], lag_ratio=0.2))
        
        # 모델 복사본이 각 클라이언트로 이동
        for i, (model_copy, client) in enumerate(zip(model_copies, self.clients)):
            client_pos = client[0].get_center() + UP * 0.6  # 클라이언트 위쪽에 위치
            self.play(
                TransformFromCopy(server_model, model_copy),
                run_time=1
            )
            self.play(
                model_copy.animate.move_to(client_pos),
                run_time=0.8
            )
        
        # 설명 텍스트
        description = Text("중앙 모델을 각 클라이언트에 배포", font_size=24, color=self.text_color)
        description.to_edge(DOWN, buff=0.5)
        
        self.play(Write(description))
        self.wait(1)
        
        # 객체 저장
        self.server_model = server_model
        self.client_models = model_copies
        self.model_arrows = arrows
        
        # 화면 정리
        self.play(
            FadeOut(description),
            FadeOut(self.model_arrows)
        )
    
    def scene3_local_training(self):
        # 제목 업데이트
        new_title = Text("2. 로컬 학습 진행", font_size=40, color=self.text_color)
        new_title.to_edge(UP, buff=0.5)
        
        self.play(
            Transform(self.title, new_title)
        )
        self.wait(0.5)
        
        # 각 클라이언트에서 로컬 학습 시각화
        updated_models = VGroup()
        
        for i, (client, model) in enumerate(zip(self.clients, self.client_models)):
            client_center = client[0].get_center()
            
            # 데이터 아이콘이 모델로 흡수되는 애니메이션
            data_icon = self.create_data_icon().scale(0.7)
            data_icon.move_to(client_center)
            
            # "학습 중" 표시
            training_text = Text("로컬 학습 중...", font_size=18, color=self.text_color)
            training_text.next_to(client, UP, buff=0.8)
            
            # 업데이트된 모델 (색상 변경)
            updated_model = self.create_model_icon(color=self.updated_model_color).scale(0.7)
            updated_model.move_to(model.get_center())
            updated_models.add(updated_model)
            
            # 애니메이션
            self.play(
                Write(training_text),
                run_time=0.5
            )
            
            # 데이터 파일이 모델을 향해 이동하는 애니메이션
            for _ in range(3):  # 각 클라이언트마다 3번의 데이터-모델 상호작용
                data_copy = data_icon.copy()
                data_copy.next_to(model, DOWN, buff=0.2)
                self.play(
                    FadeIn(data_copy), 
                    run_time=0.3
                )
                self.play(
                    data_copy.animate.move_to(model.get_center()),
                    model.animate.scale(1.05),  # 약간 확대
                    run_time=0.5
                )
                self.play(
                    FadeOut(data_copy),
                    model.animate.scale(1/1.05),  # 원래 크기로
                    run_time=0.3
                )
            
            # 모델이 업데이트된 모습으로 변화
            self.play(
                Transform(model, updated_model),
                FadeOut(training_text),
                run_time=1
            )
        
        # 전체 설명 텍스트
        description = Text("각 클라이언트는 자신의 데이터로 모델 학습", font_size=24, color=self.text_color)
        description.to_edge(DOWN, buff=0.5)
        
        self.play(Write(description))
        self.wait(1.5)
        
        # 객체 저장
        self.updated_client_models = updated_models
        
        # 화면 정리
        self.play(FadeOut(description))
        self.wait(0.5)
    
    def scene4_model_updates(self):
        # 제목 업데이트
        new_title = Text("3. 모델 업데이트 전송", font_size=40, color=self.text_color)
        new_title.to_edge(UP, buff=0.5)
        
        self.play(
            Transform(self.title, new_title)
        )
        self.wait(0.5)
        
        # 각 클라이언트에서 서버로 업데이트 전송
        delta_symbols = VGroup()
        update_arrows = VGroup()
        
        server_center = self.server[0].get_center()
        for i, (client, model) in enumerate(zip(self.clients, self.client_models)):
            client_center = client[0].get_center()
            
            # 델타 심볼 생성 (업데이트 의미)
            delta = MathTex(r"\Delta w_{" + f"{i+1}" + r"}", color=self.text_color)
            delta.scale(0.8)
            delta.move_to(model.get_center())
            delta_symbols.add(delta)
            
            # 업데이트를 서버로 보내는 화살표
            arrow = Arrow(
                start=client_center,
                end=server_center,
                buff=self.server_radius,
                stroke_width=2,
                color=self.arrow_color
            )
            update_arrows.add(arrow)
        
        # 델타 심볼 등장
        self.play(
            LaggedStart(*[FadeIn(delta) for delta in delta_symbols], lag_ratio=0.2),
            run_time=1.5
        )
        
        # 화살표 생성 및 델타 심볼이 서버로 이동
        for i, (arrow, delta) in enumerate(zip(update_arrows, delta_symbols)):
            self.play(
                GrowArrow(arrow),
                run_time=0.5
            )
            
            # 델타가 서버로 이동
            self.play(
                delta.animate.move_to(server_center),
                run_time=0.8
            )
        
        # 서버 모델 업데이트
        updated_server_model = self.create_model_icon(color=self.updated_model_color).scale(0.8)
        updated_server_model.move_to(self.server_model.get_center())
        
        self.play(
            Transform(self.server_model, updated_server_model),
            *[FadeOut(delta) for delta in delta_symbols],
            run_time=1
        )
        
        # 설명 텍스트
        description = Text("모델 업데이트만 서버로 전송 (데이터는 공유하지 않음)", font_size=24, color=self.text_color)
        description.to_edge(DOWN, buff=0.5)
        
        self.play(Write(description))
        self.wait(1.5)
        
        # 객체 저장
        self.update_arrows = update_arrows
        
        # 화면 정리
        self.play(
            FadeOut(description),
            FadeOut(self.update_arrows),
            FadeOut(self.averaging_formula)  # self.averaging_formula 대신 averaging_formula 사용
        )
    
    def scene5_redistribution(self):
        # 제목 업데이트
        new_title = Text("4. 새 모델 재배포", font_size=40, color=self.text_color)
        new_title.to_edge(UP, buff=0.5)

        self.play(
            Transform(self.title, new_title)
        )
        self.wait(0.5)

        # 새로운 화살표 - 서버에서 각 클라이언트로
        redist_arrows = VGroup()

        server_center = self.server[0].get_center()
        for i, client in enumerate(self.clients):
            client_center = client[0].get_center()

            # 화살표 생성
            arrow = Arrow(
                start=server_center,
                end=client_center,
                buff=self.server_radius,
                stroke_width=2,
                color=self.arrow_color
            )
            redist_arrows.add(arrow)

        # 화살표 등장
        self.play(
            LaggedStart(*[GrowArrow(arrow) for arrow in redist_arrows], lag_ratio=0.2),
            run_time=1.5
        )

        # 서버에서 업데이트된 모델 복사본 생성 및 이동
        new_models_temp_pos = VGroup() # 임시 위치의 새 모델 그룹
        final_model_positions = [model.get_center() for model in self.client_models] # 최종 위치 저장

        for i, client in enumerate(self.clients):
            # 업데이트된 새 모델 (서버와 같은 색)
            new_model = self.create_model_icon(color=self.updated_model_color).scale(0.7)
            new_model.move_to(server_center)

            # 새 모델이 각 클라이언트 위쪽으로 이동 (임시 위치)
            client_pos_above = client[0].get_center() + UP * 0.6
            self.play(
                TransformFromCopy(self.server_model, new_model),
                run_time=0.7
            )
            self.play(
                new_model.animate.move_to(client_pos_above),
                run_time=0.7
            )
            new_models_temp_pos.add(new_model) # 임시 위치 그룹에 추가

        # 기존 모델 사라지고 새 모델이 최종 위치로 이동
        new_client_models = VGroup() # 최종 클라이언트 모델 그룹
        fade_out_anims = [FadeOut(old_model) for old_model in self.client_models]
        move_to_final_pos_anims = []
        for i, new_model in enumerate(new_models_temp_pos):
            target_pos = final_model_positions[i]
            move_to_final_pos_anims.append(new_model.animate.move_to(target_pos))
            new_client_models.add(new_model) # 최종 그룹에 추가 (애니메이션 후 상태 반영)

        self.play(
            *fade_out_anims,
            *move_to_final_pos_anims,
            run_time=0.7
        )
        self.client_models = new_client_models # 클래스 멤버 업데이트

        # 반복 과정 표시 - 순환 화살표
        cycle_arrow = CurvedArrow(
            start_point=DOWN * 2.5 + RIGHT * 2,
            end_point=DOWN * 2.5 + LEFT * 2,
            color=self.arrow_color,
            stroke_width=3,
            angle=TAU/4
        )

        cycle_text = Text("학습 과정 반복", font_size=20, color=self.text_color)

        # 화살표와 텍스트 생성 애니메이션
        self.play(
            Create(cycle_arrow),
            run_time=0.8
        )
        # 화살표가 생성된 후 텍스트 위치 지정 및 표시
        cycle_text.next_to(cycle_arrow, DOWN, buff=0.2)
        self.play(
            Write(cycle_text),
            run_time=0.5
        )
        self.wait(1)

        # 설명 텍스트
        description = Text("통합된 모델을 다시 클라이언트에 배포", font_size=24, color=self.text_color)
        description.to_edge(DOWN, buff=1.2)

        self.play(Write(description))
        self.wait(1.5)

        # 화면 정리
        self.play(
            FadeOut(redist_arrows),
            FadeOut(description),
            FadeOut(cycle_arrow),
            FadeOut(cycle_text)
        )

    def scene6_summary(self):
        # 제목 업데이트
        new_title = Text("연합학습 요약", font_size=40, color=self.text_color)
        new_title.to_edge(UP, buff=0.5)

        self.play(
            Transform(self.title, new_title)
        )
        self.wait(0.5)

        # 요약 과정을 보여주기 위해 화면 정리
        # self.mobjects에서 title 제외하고 모두 제거 (더 명확한 방식)
        objects_to_remove = VGroup(*[m for m in self.mobjects if m != self.title])
        self.play(FadeOut(objects_to_remove))
        # self.play(
        #     *[FadeOut(obj) for obj in self.mobjects if obj != self.title]
        # ) # 이전 방식 대신 위 방식으로 변경

        # 3단계 순환 애니메이션 준비
        steps_text = [
            "1. 중앙 서버가 초기 모델 배포",
            "2. 클라이언트에서 로컬 데이터로 학습",
            "3. 모델 업데이트만 서버에 전송",
            "4. 서버에서 모델 통합 후 재배포"
        ]

        steps = VGroup()
        for i, text in enumerate(steps_text):
            step = Text(text, font_size=28, color=self.text_color)
            # 첫 번째 스텝은 중앙에 배치하고 나머지는 아래로 정렬
            if i == 0:
                 step.move_to(ORIGIN + UP * 1.5) # 초기 위치 설정
            else:
                 step.next_to(steps[i-1], DOWN, buff=0.5, aligned_edge=LEFT)
            steps.add(step)

        # 생성된 후 왼쪽 정렬 및 중앙 배치
        steps.arrange(DOWN, buff=0.5, aligned_edge=LEFT)
        steps.center().shift(UP * 0.5) # 전체적으로 약간 위로 이동

        # 단계별로 표시
        for step in steps:
            self.play(Write(step), run_time=1)
            self.wait(0.5)

        self.wait(1)

        # 최종 메시지
        final_message = Text("데이터는 로컬에, 학습은 글로벌하게", font_size=32, color=BLUE_D)
        final_message.to_edge(DOWN, buff=0.8)

        # 최종 메시지 표시 및 단계 텍스트 위로 이동
        self.play(
            Write(final_message),
            steps.animate.shift(UP * 0.5) # 메시지를 위한 공간 확보
        )

        # 연합학습의 장점 강조
        benefits = VGroup(
            Text("개인정보 보호", font_size=24, color=GREEN_D),
            Text("데이터 주권 보장", font_size=24, color=GREEN_D),
            Text("통신 비용 절감", font_size=24, color=GREEN_D)
        )

        benefits.arrange(RIGHT, buff=1)
        # final_message가 생성된 후 위치 지정
        benefits.next_to(final_message, UP, buff=0.5)

        self.play(FadeIn(benefits))

        self.wait(2)

Manim Community v0.19.0

Exception: Cannot call Mobject.point_from_proportion for a Mobject with no points